In [1]:
import wandb

wandb.login()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: andtun (obuchii). Use `wandb login --relogin` to force relogin


True

In [2]:
import pandas as pd

MODEL_LIST = pd.read_csv("model_list.csv").set_index("index")
MODEL_LIST

,model,price
index,,
gpt4o,gpt-4o-2024-08-06,2.8800
gpt4omini,gpt-4o-mini,0.1728
gpt4turbo,gpt-4-turbo,8.6400
gpt4,gpt-4,17.2800
gpt35,gpt-3.5-turbo-0125,0.4320


In [3]:
import json
import os
from openai import OpenAI
from dotenv import load_dotenv
from typing import List, Tuple, Dict, Union

# Загрузка переменных окружения из .env файла
load_dotenv()


class GPT4SpellChecker:
    """
    Класс для проверки орфографии и пунктуации с использованием GPT-4 через OpenAI API.

    Attributes:
        api_key (str): Ключ для доступа к API OpenAI.
    """

    BASE_URL = "https://api.proxyapi.ru/openai/v1"

    SYSTEM_PROMPT = "Ты опытный корректор текста."

    PROMPT_TEMPLATE = """
Ты выступаешь в роли профессионального корректора текста на русском языке с многолетним опытом. Твоя задача состоит в следующем:

Проанализировать и исправить все орфографические, грамматические и пунктуационные ошибки в тексте, сохраняя исходный смысл и стиль.
Для каждой ошибки:
Определи её точное местоположение, указав индекс начала ошибки.
Укажи текст ошибки.
Предложи один или несколько вариантов исправления.
Объясни, почему это ошибка, и укажи её тип (орфографическая, грамматическая, пунктуационная).
Верни результат в формате JSON, где для каждой ошибки укажи:
"index": индекс начала ошибки в тексте,
"error": исходный текст с ошибкой,
"suggestions": список предложенных исправлений,
"message": краткое описание ошибки и её типа.
После исправлений предоставь исправленный текст.

Пример 1: Оригинальный текст: "Я обожаю учится, это делаэт меня лучше!" Результат:

{
    "corrections": [
        {
            "index": 9,
            "error": "учится",
            "suggestions": ["учиться"],
            "message": "Грамматическая ошибка: инфинитив 'учиться' должен использоваться в данном контексте."
        },
        {
            "index": 19,
            "error": "делаэт",
            "suggestions": ["делает"],
            "message": "Орфографическая ошибка: глагол 'делает' пишется через 'е'."
        }
    ],
    "corrected_text": "Я обожаю учиться, это делает меня лучше!"
}
Пример 2: Оригинальный текст: "Пока я небыл дома, ктото пришел." Результат:

{
    "corrections": [
        {
            "index": 8,
            "error": "небыл",
            "suggestions": ["не был"],
            "message": "Грамматическая ошибка: частица 'не' пишется отдельно от глагола."
        },
        {
            "index": 18,
            "error": "ктото",
            "suggestions": ["кто-то"],
            "message": "Орфографическая ошибка: слово 'кто-то' пишется через дефис."
        }
    ],
    "corrected_text": "Пока я не был дома, кто-то пришел."
}
Используй этот формат и исправь следующий текст: "%s"
    """
    corrections_split_prefix = "2. Список исправлений:"

    def __init__(self, model_name: str, temperature: float = 0.7):
        """
        Инициализация модели GPT-4 через OpenAI API.

        Args:
            model_name (str): Название модели, по умолчанию используется GPT-4.
            temperature (float): Температура модели для управления степенью творчества.
        """
        # Получение API ключа из переменной окружения
        api_key = os.getenv("OPENAI_API_KEY")
        if api_key is None:
            raise ValueError("API ключ OpenAI не найден в переменных окружения.")

        self.client = OpenAI(
            api_key=api_key,
            base_url=self.BASE_URL,
        )

        self.model_name = model_name
        self.temperature = temperature

    def predict_verbose(
        self, text: str
    ) -> Tuple[List[Dict[str, Union[str, float]]], str]:
        """
        Возвращает исправленный текст и список всех предложенных исправлений с типом ошибки и уверенностью.

        Args:
            text (str): Входной текст для проверки.

        Returns:
            Tuple[List[Dict[str, Union[str, float]]], str]: Исправленный текст и список всех предложенных исправлений.
        """
        prompt = self._create_prompt(text)

        response = self.client.chat.completions.create(
            model=self.model_name,
            messages=[
                {"role": "system", "content": self.SYSTEM_PROMPT},
                {"role": "user", "content": prompt},
            ],
            temperature=self.temperature,
        )

        # Получаем ответ от модели
        result = response.choices[0].message.content

        r = self._process_gpt4_output(result)
        return r["corrections"], r["corrected_text"]

    def _create_prompt(self, text: str) -> str:
        """
        Создает промпт для GPT-4, чтобы обеспечить максимально качественное исправление текста с уверенностью и типом ошибки.

        Args:
            text (str): Входной текст для проверки.

        Returns:
            str: Промпт для модели GPT-4.
        """
        return self.PROMPT_TEMPLATE % text

    def _process_gpt4_output(
        self, result: str
    ) -> Dict[str, Union[List[Dict[str, str]], str]]:
        """
        Обрабатывает результат, полученный от модели GPT-4.

        Args:
            result (str): Результат, возвращаемый GPT-4.

        Returns:
            Dict[str, Union[List[Dict[str, str]], str]]: Словарь с исправлениями и исправленным текстом.
        """
        # Ожидаем, что GPT-4 вернет результат в формате JSON
        try:
            result = result.replace("```json", "").replace("```", "")
            result_json = json.loads(result)
        except Exception as e:
            raise ValueError(f"Ошибка при парсинге ответа: {e}")

        return result_json

    def predict(self, text: str) -> str:
        """
        Возвращает исправленный текст.

        Args:
            text (str): Входной текст для проверки.

        Returns:
            str: Исправленный текст.
        """
        _, corrected_text = self.predict_verbose(text)
        return corrected_text

In [4]:
model_name = "gpt4omini"
m = GPT4SpellChecker(model_name=MODEL_LIST.loc[model_name].model, temperature=0.2)

In [5]:
PROJECT_NAME = "gpt_spellers_100"

In [6]:
from src.datasets import load_datasets

orpho_dataset, punct_dataset = load_datasets()

/Users/andtunes/Desktop/projects/Obuch2i-NLP/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [7]:
def head(dataset, n=5):
    return dataset.select(range(n))

In [8]:
from typing import Literal
from src.model_scorers import WandbSageModelScorer


def _score_model(mode: str, dataset, model_name: str, temperature: float):
    sms = WandbSageModelScorer(dataset, project=PROJECT_NAME, run_suffix=mode)
    model = GPT4SpellChecker(model_name, temperature=temperature)
    scoring_final_result, explanation = sms.score_explain(
        model, metrics=["errant", "ruspelleval"]
    )
    wandb.run.summary["temperature"] = temperature
    wandb.run.summary["prompt"] = model.PROMPT_TEMPLATE
    wandb.run.summary["model_name"] = model_name
    return scoring_final_result, explanation


def score(
    mode: Literal["orpho"] | Literal["punct"],
    model_name,
    temperature: float,
    n_samples=5,
):
    if mode == "orpho":
        _score_model(
            mode="orpho",
            dataset=head(orpho_dataset["test"], n=n_samples),
            model_name=model_name,
            temperature=temperature,
        )
    elif mode == "punct":
        _score_model(
            mode="punct",
            dataset=head(punct_dataset["test"], n=n_samples),
            model_name=model_name,
            temperature=temperature,
        )
    else:
        raise ValueError("No such mode")

# Тестирование: орфография и пунктуация

In [9]:
N_SAMPLES = 100

for model_name in MODEL_LIST.model:
    score("punct", model_name, temperature=0.2, n_samples=N_SAMPLES)
    score("punct", model_name, temperature=0.7, n_samples=N_SAMPLES)

 17%|█▋        | 17/100 [01:00<05:04,  3.67s/it]

Ошибка при парсинге ответа: Extra data: line 27 column 1 (char 857)


 48%|████▊     | 48/100 [03:02<04:07,  4.77s/it]

Ошибка при парсинге ответа: Extra data: line 21 column 1 (char 598)


 52%|█████▏    | 52/100 [03:20<03:42,  4.64s/it]

...

Error code: 402 - {'detail': 'Insufficient balance to run this request.'}


100%|██████████| 100/100 [01:45<00:00,  1.05s/it]


Error code: 402 - {'detail': 'Insufficient balance to run this request.'}


Calculating words metric: 100%|██████████| 100/100 [00:00<00:00, 1116.58it/s]


ZeroDivisionError: division by zero

In [ ]:
wandb.run.finish()

prompt,Ты выступаешь в рол...
temperature,0.2


# Выводы

1. (см [на WandB](https://wandb.ai/obuchii/gpt_spellers_100/table)) ChatGPT показывает результат хуже чем трансформеры
2. При этом ChatGPT может возвращать тип ошибок
3. Вероятно, если дообучить ChatGPT - он покажет качество сравнимое или лучше чем трансформеры
4. **Стоит проэкспериментировать и с другими промптами**
5. Одна полная итерация эксперимента будет стоить около 5к рублей

ToDo:

1. Попросить модель выводить только исправленный текст - сэкономим на токенах, но потеряем в качестве
2. Сократить систем промпт и юзер промпт
3. Менять местами систем промпт и юзер промпт, экспериментировать с гиперпараметрами
4. **Тестировать модели только на датасете punct (потому что орф ошибки он тоже содержит) - для экономии токенов**